In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import keras.backend as K

IMAGE_SHAPE = [94,24]
CHARS = "ABCDEFGHIJKLMNPQRSTUVWXYZ0123456789" # exclude I, O
CHARS_DICT = {char:i for i, char in enumerate(CHARS)}
DECODE_DICT = {i:char for i, char in enumerate(CHARS)}
NUM_CLASS = len(CHARS)+1



In [13]:
def CTCLoss(y_true, y_pred):
    # Compute the training-time loss value
    batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
    input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
    label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

    input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
    label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

    loss = keras.backend.ctc_batch_cost(y_true, y_pred, input_length, label_length)
    
    return loss

In [14]:
class small_basic_block(keras.layers.Layer):

    def __init__(self,out_channels,name=None,**kwargs):
        super().__init__(**kwargs)
        out_div4=int(out_channels/4)
        self.main_layers = [
            keras.layers.Conv2D(out_div4,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Conv2D(out_div4,(3,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Conv2D(out_div4,(1,3),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Conv2D(out_channels,(1,1),padding='same',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]  
    
    def call(self,input):
        x = input
        for layer in self.main_layers:
            x = layer(x)
        return x

In [15]:
#test this later

class global_context(keras.layers.Layer):
    def __init__(self,kernel_size,stride,**kwargs):
        super().__init__(**kwargs)
        self.ksize = kernel_size
        self.stride = stride


    def call(self, input):
        x = input 
        avg_pool = keras.layers.AveragePooling2D(pool_size=self.ksize,strides=self.stride,padding='same')(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(avg_pool)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        out = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([avg_pool , sqm])
        #out = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(avg_pool)
        return out

In [16]:
class LPRnet(keras.Model):
    def __init__(self, input_shape=(24,94,3), **kwargs):
        super(LPRnet, self).__init__(**kwargs)
        self.input_layer = keras.layers.Input(input_shape)
        self.cnn_layers= [
            keras.layers.Conv2D(64,kernel_size = (3,3),strides=1,padding='same',name='main_conv1',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(name='BN1'),
            keras.layers.ReLU(name='RELU1'),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,1),name='maxpool2d_1',padding='same'),
            small_basic_block(128),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_2',padding='same'),
            small_basic_block(256),
            small_basic_block(256),
            keras.layers.MaxPool2D(pool_size=(3,3),strides=(1,2),name='maxpool2d_3',padding='same'),
            keras.layers.Dropout(0.5),
            keras.layers.Conv2D(256,(4,1),strides=1,padding='same',name='main_conv2',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
            keras.layers.Dropout(0.5),
            keras.layers.Conv2D(NUM_CLASS,(1,13),padding='same',name='main_conv3',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),  
            keras.layers.BatchNormalization(),
            keras.layers.ReLU(),
        ]
        self.out_layers = [
            keras.layers.Conv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out',kernel_initializer=keras.initializers.glorot_uniform(),bias_initializer=keras.initializers.constant()),
            #keras.layers.BatchNormalization(),
            #keras.layers.ReLU(),
        ]
        #self.conv_out = keras.layers.Conv2D(NUM_CLASS,kernel_size=(1,1),strides=(1,1),padding='same',name='conv_out')
        self.out = self.call(self.input_layer)
        super(LPRnet, self).__init__(
            inputs=self.input_layer,
            outputs=self.out,
            **kwargs)

    def call(self,inputs,training=False):
        x = inputs
        layer_outputs = []
        for layer in self.cnn_layers:
            x = layer(x)
            layer_outputs.append(x)
        scale1 = global_context((1,4),(1,4))(layer_outputs[0])
        scale2 = global_context((1,4),(1,4))(layer_outputs[4])
        scale3 = global_context((1,2),(1,2))(layer_outputs[6])
        scale5 = global_context((1,2),(1,2))(layer_outputs[7])
        #scale4 = keras.layers.Lambda(lambda x: K.l2_normalize(x,axis=1))(x)
        sq = keras.layers.Lambda(lambda x: tf.math.square(x))(x)
        sqm = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x))(sq)
        scale4 = keras.layers.Lambda(lambda x: tf.math.divide(x[0], x[1]))([x , sqm])
        gc_concat = keras.layers.Lambda(lambda x: tf.concat([x[0], x[1], x[2], x[3], x[4]],3))([scale1, scale2, scale3, scale5,scale4])
        for layer in self.out_layers:
            gc_concat = layer(gc_concat)
        logits = keras.layers.Lambda(lambda x: tf.math.reduce_mean(x[0],axis=1))([gc_concat])
        #transposed_logs = keras.layers.Lambda(lambda x: tf.transpose(x,(1,0,2)))(logits)
        logits = keras.layers.Softmax()(logits)
        return logits

In [17]:
model = LPRnet()

In [18]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=2000,
    decay_rate=0.9
)

In [ ]:


model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),loss =CTCLoss)
for layer in model.layers:
    print(layer.name,layer.output_shape)

In [ ]:
model.summary()

In [27]:
import glob
import cv2
data = []
labels = []
val_data = []
val_labels = []

GEN_VAL_PATH = glob.glob('./valid/*.jpg')
LP_VAL_PATH = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\filtered_plates\\driving_quezoncity.mp4\\*.png')

#for file in glob.glob('./test/*.jpg'):
#    label=file.split('\\')[1]
#    image = cv2.imread(file,cv2.IMREAD_COLOR)
#    image = cv2.resize(image,(94,24))
#    data.append(image/256)
#    labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
for file in LP_VAL_PATH:
    label=file.split('\\')[-1]
    image = cv2.imread(file,cv2.IMREAD_COLOR)
    image = cv2.resize(image,(94,24))
    val_data.append(image/256)
    val_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])


In [ ]:
#training_set = np.array(data,dtype=np.float32)
#training_labels = np.array(labels)
#ragged = tf.ragged.constant(training_labels).to_tensor()
#dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).repeat(100).batch(64)


val_training_set = np.array(val_data,dtype=np.float32)
val_training_labels = np.array(val_labels)
val_ragged = tf.ragged.constant(val_training_labels).to_tensor()
val_dataset = tf.data.Dataset.from_tensor_slices((val_training_set,val_ragged)).shuffle(640).repeat(100).batch(64)

In [12]:
from gen_plates_keras import *
gen = ImageGenerator()
def generate_dataset(num = 100):
    data, labels = gen.generate_images(num)
    gen_labels = []
    for label in labels:
        gen_labels.append([CHARS_DICT[i] for i in label.split('_')[0]])
    pics =np.array(data)
    labels = np.array(labels)
    training_set = np.array(pics,dtype=np.float32)
    training_labels = np.array(gen_labels)
    ragged = tf.ragged.constant(training_labels).to_tensor()
    dataset = tf.data.Dataset.from_tensor_slices((training_set,ragged)).shuffle(640).batch(64)
    return dataset

standalone_dataset = generate_dataset(num=100)

C:\Users\carlos\AppData\Local\Temp/ipykernel_21524/3666392512.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training_labels = np.array(gen_labels)


In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="my-test-project", entity="clsandoval")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 400,
  "batch_size": 64
}

def generate_val_set(num=400):
        val_data_gen, val_labels_gen = gen.generate_images(num)
        val_labels_encoded = []
        val_data_encoded = []
        for image,label in zip(val_data_gen, val_labels_gen):
                val_data_encoded.append(np.expand_dims(image,axis=0))
                val_labels_encoded.append([CHARS_DICT[i] for i in label.split('_')[0]])
        return val_data_encoded,val_labels_encoded
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
#standalone_dataset = generate_dataset(num=10000)
#val_dataset = generate_dataset(1000)
for j in range(100):
    dataset = generate_dataset(num=32000)
    model.fit(dataset,validation_data=val_dataset,validation_steps=5,epochs=10,steps_per_epoch=50,callbacks=[WandbCallback()])
    print(j,flush=True)

In [ ]:
import glob
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\filtered_plates\\driving_quezoncity.mp4\\*.png')
for file in real_images:
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    preds = model.predict(test_image) 
    decoded = tf.keras.backend.ctc_decode(preds,(24,))
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            print(DECODE_DICT[i],end='')
    print(" "+ file.split('\\')[-1].split('_')[0])

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('keras_lprnet2.tflite', 'wb') as f:
  f.write(tflite_model)

In [3]:
interpreter = tf.lite.Interpreter('keras_lprnet1.tflite')

In [4]:
import numpy as np
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
real_images = glob.glob('C:\\Users\\carlos\\Desktop\\cs\\ml-sandbox\\ANPR\\TFODCourse\\test frames\\driving_quezoncity.mp4\\*.png')
for file in real_images:
    image = cv2.imread(file)
    test_image = cv2.resize(image,(94,24))/256
    test_image = np.expand_dims(test_image,axis=0)
    test_image = test_image.astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    decoded = keras.backend.ctc_decode(output_data,(24,))
    for i in np.array(decoded[0]).reshape(24):
        if i >-1:
            print(DECODE_DICT[i],end='')
    print(" "+ file.split('\\')[-1].split('_')[0])
